# Парсинг цен криптовалют

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Парсинг

In [2]:
# Сайт для парсинга и данные браузера
URL = 'https://coinmarketcap.com/currencies/'
HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                         '(KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36',
           'accept': '*/*'}

In [3]:
# Функция парсинга
def get_html(url, params=None):
    r = requests.get(url, headers=HEADERS, params=params)
    return r

In [4]:
# Функция для распарсивания
def get_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_='top-summary-container')

    crypto = []
    for item in items:
        # в список crypto добавляем тикер и цену в долларах для криптовалюты 
        crypto.append({
            'ticker'   : item.find('small', class_='nameSymbol').get_text(),
            'usd_price': item.find('div', class_='priceValue').get_text().replace('$','').replace(',','')
        })

    return crypto

In [5]:
def parse():
    lst = []
    for coin in df.coin:
        coin = coin.replace(' ', '-')
        # создаем ссылку, с которой будем парсить
        html = get_html(URL + f'{coin}')
        if html.status_code == 200:
            price_ticker = get_content(html.text)
            lst.extend(price_ticker)
        else:
            print('Error')
    return lst

# Read table from gspread

In [6]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [9]:
# Specify path to your file with credentials
path_to_credential = '***.json'

# Specify name of table in google sheets
table_name = 'my_crypto'

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_credential, scope)

gs = gspread.authorize(credentials)
work_sheet = gs.open(table_name)

# Select 1st sheet
sheet1 = work_sheet.sheet1

# Get data in python lists format
data = sheet1.get_all_values()

# Get header from data
headers = data.pop(0)

# Create df
df = pd.DataFrame(data, columns=headers)
df

,coin,ticker,purchase_value,amount
0,Chiliz,CHZ,33.11,110.81
1,Dash,DASH,55.96,0.29394399
2,Dia,DIA,34.15,17.7937
3,EOS,EOS,55.03,9.1476778
4,Ethereum,ETH,58.23,0.02588
5,Siacoin,SC,35.75,1799
6,TRON,TRX,50.72,580.703
7,Verge,XVG,48.54,1379.62


# Parse prices

In [10]:
#запускаем парсинг всех монет из нашей таблицы
parser_table = parse()

## Работа с таблицами

In [11]:
# Делаем DataFrame из списка словарей, полученного при парсинге
parser_table = pd.DataFrame(parser_table)

In [12]:
# Соединяем нашу таблицу и полученный DataFrame по столбцу 'ticker'
full_df = df.merge(parser_table, how='left', on='ticker')

# Заменяем запятые на точки в колонках 'purchase_value' и 'amount'
full_df.purchase_value = full_df.purchase_value.str.replace(',', '.')
full_df.amount = full_df.amount.str.replace(',', '.')

# Меняем тип данных в столбцах 'purchase_value', 'amount' и 'usd_price' на float
full_df = full_df.astype({'purchase_value': 'float', 'amount': 'float', 'usd_price': 'float'})

# Создаем столбец 'current_value' со стоимостью каждой монеты в портфеле в $
full_df['current_value'] = (full_df.amount * full_df.usd_price).round(2)

# Округляем текущую цену криптовалюты до 3 знаков после запятой
full_df.usd_price = full_df.usd_price.round(3)

# Считаем изменение в % для каждой из криптовалют
full_df['profit'] = (full_df.current_value / full_df.purchase_value * 100 - 100).round(2)

full_df

,coin,ticker,purchase_value,amount,usd_price,current_value,profit
0,Chiliz,CHZ,33.11,110.810000,0.334,36.98,11.69
1,Dash,DASH,55.96,0.293944,191.720,56.35,0.70
2,Dia,DIA,34.15,17.793700,1.800,32.03,-6.21
3,EOS,EOS,55.03,9.147678,4.730,43.27,-21.37
4,Ethereum,ETH,58.23,0.025880,3595.970,93.06,59.81
5,Siacoin,SC,35.75,1799.000000,0.022,39.67,10.97
6,TRON,TRX,50.72,580.703000,0.102,59.12,16.56
7,Verge,XVG,48.54,1379.620000,0.026,36.22,-25.38


In [13]:
# Итоговая стоимость портфеля в $
total_value = full_df.current_value.sum()

# Первоначальные вложения
total_purchase = 220.94 + 40 + 42.6

# Итоговое изменение портфеля в %
total_profit = round((total_value / total_purchase * 100 - 100), 2)

# Создаем DataFrame с итогами по портфелю
total_df = pd.DataFrame({'Криптовалюта': 'Итого', 
                         'Вложения, $': total_purchase,
                         'Стоимость, $': total_value, 
                         'Изменение, %': total_profit}, index=['-'])

total_df

,Криптовалюта,"Вложения, $","Стоимость, $","Изменение, %"
-,Итого,303.54,396.7,30.69


In [14]:
# Меняем название колонок на более понятные
full_df = full_df.rename(columns={'coin': 'Криптовалюта', 
                                'ticker': 'Тикер', 
                                'purchase_value': 'Вложения, $', 
                                'amount': 'Количество', 
                                'usd_price': 'Цена, $',
                                'current_value': 'Стоимость, $',
                                'profit': 'Изменение, %'
                                })

# Добавляем итоги по портфелю с заполнением пустых значений '-'
full_df = full_df.append(total_df).fillna('-')

In [15]:
full_df

,Криптовалюта,Тикер,"Вложения, $",Количество,"Цена, $","Стоимость, $","Изменение, %"
0,Chiliz,CHZ,33.11,110.81,0.334,36.98,11.69
1,Dash,DASH,55.96,0.293944,191.72,56.35,0.70
2,Dia,DIA,34.15,17.7937,1.8,32.03,-6.21
3,EOS,EOS,55.03,9.147678,4.73,43.27,-21.37
4,Ethereum,ETH,58.23,0.02588,3595.97,93.06,59.81
5,Siacoin,SC,35.75,1799.0,0.022,39.67,10.97
6,TRON,TRX,50.72,580.703,0.102,59.12,16.56
7,Verge,XVG,48.54,1379.62,0.026,36.22,-25.38
-,Итого,-,303.54,-,-,396.70,30.69


In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter("Криптовалюта.xlsx", engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
full_df.to_excel(writer, index=False, sheet_name='Sheet1')

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']


# Set the column width.
worksheet.set_column('A:H', 15)

# Close the Pandas Excel writer and output the Excel file.
writer.save()